# Python Program for Calculating Call or Put Buying Total

In [121]:
#importing Imp Libraries 

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import datetime, time, math

start_time = time.time()

# Create a session and load the page
chromedriver_path = 'chromedriver.exe' # Change it to your own chromedriver path!
driver = webdriver.Chrome(executable_path=chromedriver_path)
driver.get('https://www1.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp')

# Wait for the page to fully load
driver.implicitly_wait(5)

# Parse HTML code and grab tables with Beautiful Soup
soup = BeautifulSoup(driver.page_source, 'lxml')

tables = soup.find_all('table')

# Read tables with Pandas read_html()
dfs = pd.read_html(str(tables))

#print(f'Total tables: {len(dfs)}')
#print(dfs[0])

In [122]:
#our main Table 
#----------------
dfs[1:]

[     CALLS                                                             \
      Chart        OI Chng in OI   Volume     IV   LTP Net Chng BidQty   
 0      NaN         -          -        -      -     -        -      -   
 1      NaN         -          -        -      -     -        -      -   
 2      NaN         -          -        -      -     -        -      -   
 3      NaN         -          -        -      -     -        -      -   
 4      NaN         -          -        -      -     -        -      -   
 ..     ...       ...        ...      ...    ...   ...      ...    ...   
 136    NaN     12450       5025      281  37.88  0.85     0.20    300   
 137    NaN      2400       2400       32  43.74  2.25   -14.60     75   
 138    NaN     12825        375      375  40.41  0.85     0.15    525   
 139    NaN       450        300        4  41.66  0.95    -0.90    150   
 140  Total  43005750        NaN  1116353    NaN   NaN      NaN    NaN   
 
                        ...     PUTS

In [123]:
#create an empty df
df = pd.DataFrame(dfs[1:]) 

In [124]:
#copy table into df
df = dfs[1]
#----------------
df

CALLS                                                             \
     Chart        OI Chng in OI   Volume     IV   LTP Net Chng BidQty   
0      NaN         -          -        -      -     -        -      -   
1      NaN         -          -        -      -     -        -      -   
2      NaN         -          -        -      -     -        -      -   
3      NaN         -          -        -      -     -        -      -   
4      NaN         -          -        -      -     -        -      -   
..     ...       ...        ...      ...    ...   ...      ...    ...   
136    NaN     12450       5025      281  37.88  0.85     0.20    300   
137    NaN      2400       2400       32  43.74  2.25   -14.60     75   
138    NaN     12825        375      375  40.41  0.85     0.15    525   
139    NaN       450        300        4  41.66  0.95    -0.90    150   
140  Total  43005750        NaN  1116353    NaN   NaN      NaN    NaN   

                       ...     PUTS                                         \
    BidPrice AskPrice  ... BidPrice AskPrice AskQty Net Chng   LTP      IV   
0          -        -  ...     0.30     1.00    675    -0.75  0.20  354.91   
1          -        -  ...     0.30     0.85    300     0.15  0.50  370.78   
2          -        -  ...     0.20     1.20    300        -     -       -   
3          -        -  ...     0.35     0.80   1500     0.35  0.85       -   
4          -        -  ...     0.15     1.30   3000        -  1.00       -   
..       ...      ...  ...      ...      ...    ...      ...   ...     ...   
136     0.85     0.90  ...  1228.65  1283.85   2475        -     -       -   
137     0.80     1.10  ...  1278.00  1333.25   2475        -     -       -   
138     0.85     0.90  ...  1329.95  1356.15     75        -     -       -   
139     0.60     0.95  ...  1373.75  1428.90   4875        -     -       -   
140      NaN      NaN  ...      NaN      NaN    NaN      NaN   NaN     NaN   

                                          
      Volume Chng in OI        OI  Chart  
0          3         75      1725    NaN  
1          4        150     26850    NaN  
2          -          -       675    NaN  
3          2          -     50175    NaN  
4          1        -75      3600    NaN  
..       ...        ...       ...    ...  
136        -          -        75    NaN  
137        -          -         -    NaN  
138        -          -         -    NaN  
139        -          -         -    NaN  
140  1160679        NaN  60733575  Total  

[141 rows x 23 columns]

In [125]:
#saving df into csv
n=str(datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S'))
name='NSE'+n+'.csv'
df.to_csv(name) 

In [126]:
#getting strick price from web page
strick_price = driver.find_element_by_xpath('//*[@id="wrapper_btm"]/table[1]/tbody/tr/td[2]/div/span[1]/b')
strick_price.text

'NIFTY 13831.50'

In [127]:
LTP=(strick_price.text[6:])
#print(type(LTP))
LTP.strip()
LTP=int(float(LTP))
print(LTP)

13831


In [128]:
#logic for strick price conversion of multiple of 150 -50
f3digit=LTP//100
print("First 3 digit",f3digit)

num=LTP//10
last=num%10
print("Last: ",last)
if last>=5:
    f3digit=(f3digit+1)*100
else:
    f3digit=(f3digit)*100
    f3digit+=50
    
print(f3digit)

First 3 digit 138
Last:  3
13850


In [129]:
#drop unneccessary columns from df
df.drop(df.columns[[0,1,4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,22]], axis = 1, inplace = True) 
#df.head() 

In [130]:
#df.columns

In [131]:
df.columns = df.columns.droplevel() #it drops multilevel indexing

In [132]:
#df.head()

In [133]:
#giving proper names to our new columns
df.columns = ['Call_Chng_in_OI', 'Call_Volume', 'Strike Price', 'Put_Chng_in_OI', 'Put_OI']

In [134]:
 #it gives index of matched column value in int form
i=df.loc[df['Strike Price']==f3digit].index.values.astype(int)[0]
#i

In [135]:
start=i-5
df2=df.iloc[start:start+11,]

In [136]:
#df2.columns

In [137]:
#df2.head()

In [138]:
#df2.info()

In [139]:
#saving df into csv
n=str(datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S'))
name='ST_NSE'+n+'.csv'
df2.to_csv(name) 

In [140]:
try:
    c_col=(df2['Call_Chng_in_OI'].astype(str).astype(int)).sum()
   # print(c_col)
    p_col=(df2['Put_Chng_in_OI'].astype(str).astype(int)).sum()
  #  print(p_col)
except Exception as e:
    c_col=0
    print('Ecxeption')
    

In [141]:
if c_col>=p_col:
    signal="Buy a PE Call"
    print("Buy a PE Call")
else:
    signal="Buy a CE Call"
    print("Buy a CE Call")

Buy a CE Call


In [142]:
#Save Every points Text File
#File Name= Output

fname='output.txt'
stltp='\n'+"Strick Price LTP: "+str(LTP)+'\n'
stltp_="SP on Data: "+str(f3digit)+'\n'
CE_PE_DATA="CE Data: "+str(c_col)+" PE DATA: "+str(p_col)+'\n'
signal1='Signal:'+signal+'\n'
diff=p_col-c_col
diff=str(diff)+'\n'

sec=((time.time() - start_time))
time_req="Required Seconds for Exe:" + str(math.trunc(sec))+'\n'
with open(fname,'a') as f:
    f.write(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    f.write(stltp)
    f.write(stltp_)
    f.write(CE_PE_DATA)
    f.write(signal1)
    f.write(diff)
    f.write(time_req)
    f.write("-------------++++++++--------------\n")
    del signal1
print("Data Updated Succesfully...!")
print("--- %s seconds ---" % math.trunc(time.time() - start_time))
time.sleep(15)
driver.close()

Data Updated Succesfully...!
--- 23 seconds ---


# SGX Nifty Data

# Bank Nifty Last five days Open Close Data